<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания №24


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

Описание задачи:
Создать базовый класс Notification в C#, который будет представлять уведомления
пользователям. На основе этого класса разработать 2-3 производных класса,
демонстрирующих принципы наследования и полиморфизма. В каждом из классов
должны быть реализованы новые атрибуты и методы, а также переопределены
некоторые методы базового класса для демонстрации полиморфизма.

Требования к базовому классу Notification:
• Атрибуты: ID уведомления (NotificationId), Текст уведомления (MessageText),
Тип уведомления (Type).
• Методы:
o DisplayNotification(): метод для отображения уведомления
пользователю.
o SendNotification(): метод для отправки уведомления.
o GetNotificationDetails(): метод для получения деталей уведомления.

Требования к производным классам:
1. EmailУведомление (EmailNotification): Должно содержать дополнительные
атрибуты, такие как Адрес электронной почты (EmailAddress).
Метод SendNotification() должен быть переопределен для отправки
уведомления по электронной почте.
2. SMSУведомление (SMSNotification): Должно содержать дополнительные
атрибуты, такие как Номер телефона (PhoneNumber).
Метод SendNotification() должен быть переопределен для отправки
уведомления через SMS.
3. PushУведомление (PushNotification) (если требуется третий класс): Должно
содержать дополнительные атрибуты, такие как Платформа (Platform,
например, iOS или Android). Метод DisplayNotification() должен быть
переопределен для отображения уведомления на мобильной платформе.

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) и реализуйте полиморфизм с перекрытием и перегрузкой методов, а также generic классы

<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [1]:
using System;
using System.Collections.Generic;
using System.Linq;

public enum ImportanceLevel { Low, Medium, High, Critical }

public abstract class Notification
{
    public int NotificationId { get; set; }
    public string MessageText { get; set; }
    public string Type { get; set; }
    public DateTime CreatedAt { get; set; }
    public bool IsRead { get; set; }
    public string Priority { get; set; }
    public string Recipient { get; set; }
    public string Sender { get; set; }
    public string Status { get; set; }

    // Новые атрибуты
    public List<string> Tags { get; set; } = new List<string>();
    public DateTime? Expiration { get; set; }
    public int RetryCount { get; set; } = 0;
    public Dictionary<string, string> Metadata { get; set; } = new Dictionary<string, string>();
    public ImportanceLevel Importance { get; set; } = ImportanceLevel.Medium;

    public Notification() { }

    public Notification(int notificationId, string messageText, string type, string priority, string recipient, string sender, string status)
    {
        NotificationId = notificationId;
        MessageText = messageText;
        Type = type;
        CreatedAt = DateTime.Now;
        IsRead = false;
        Priority = priority;
        Recipient = recipient;
        Sender = sender;
        Status = status;
    }

    public virtual void DisplayNotification()
    {
        Console.WriteLine($"[Уведомление] {MessageText} (Приоритет: {Priority}, Получатель: {Recipient}, Статус: {Status})");
    }

    // Перегрузка
    public virtual void DisplayNotification(bool showMetadata)
    {
        DisplayNotification();
        if (showMetadata && Metadata.Any())
        {
            Console.WriteLine("Метаданные:");
            foreach (var kv in Metadata)
                Console.WriteLine($" - {kv.Key}: {kv.Value}");
        }
    }

    // Перегрузка
    public virtual void DisplayNotification(string format)
    {
        if (format == "короткий")
            Console.WriteLine($"[{Type}] {MessageText}");
        else
            DisplayNotification();
    }

    public virtual void SendNotification()
    {
        Console.WriteLine("Уведомление отправлено.");
        IsRead = true;
        Status = "Отправлено";
    }

    // Перегрузка
    public virtual void SendNotification(bool async, int maxRetries = 1)
    {
        if (async) Console.WriteLine("Отправка уведомления в фоне...");
        int attempts = 0;
        while (attempts < maxRetries)
        {
            attempts++;
            SendNotification();
            if (Status.Contains("Отправлен")) break;
            RetryCount++;
        }
    }

    public virtual string GetNotificationDetails()
    {
        return $"ID: {NotificationId}, Тип: {Type}, Текст: {MessageText}, Время: {CreatedAt}, Прочитано: {IsRead}, Приоритет: {Priority}, Получатель: {Recipient}, Отправитель: {Sender}, Статус: {Status}";
    }

    // Перегрузка
    public virtual string GetNotificationDetails(bool includeMetadata)
    {
        var baseDetails = GetNotificationDetails();
        if (!includeMetadata || !Metadata.Any()) return baseDetails;
        var meta = string.Join(", ", Metadata.Select(kv => $"{kv.Key}={kv.Value}"));
        return baseDetails + $", Метаданные: {meta}";
    }

    public void MarkAsRead()
    {
        IsRead = true;
        Status = "Прочитано";
    }

    public void ChangePriority(string newPriority)
    {
        Priority = newPriority;
    }

    public void ChangeRecipient(string newRecipient)
    {
        Recipient = newRecipient;
    }

    public void ChangeStatus(string newStatus)
    {
        Status = newStatus;
    }

    // Новый метод
    public void ScheduleSend(DateTime when)
    {
        Console.WriteLine($"Уведомление {NotificationId} запланировано на {when}.");
        Metadata["ScheduledAt"] = when.ToString("o");
    }

    public virtual bool RetrySend()
    {
        RetryCount++;
        Console.WriteLine($"Повторная попытка отправки ({RetryCount}) для {NotificationId}");
        SendNotification();
        return Status.Contains("Отправлен");
    }

    public string SerializeMetadata()
    {
        return string.Join(";", Metadata.Select(kv => $"{kv.Key}={kv.Value}"));
    }
}

public interface ILoggable
{
    void Log();
}

public interface IArchivable
{
    void Archive();
}

public class EmailNotification : Notification, ILoggable, IArchivable
{
    public string EmailAddress { get; set; }
    public string Subject { get; set; }
    public string CC { get; set; }
    public int AttachmentsCount { get; set; }

    // Новые атрибуты
    public bool IsHtml { get; set; } = false;
    public string ReplyTo { get; set; }
    public int ImportanceScore { get; set; } = 5;

    public EmailNotification() { }

    public EmailNotification(int notificationId, string messageText, string emailAddress, string subject, string cc, int attachmentsCount, string priority, string recipient, string sender, string status)
        : base(notificationId, messageText, "Email", priority, recipient, sender, status)
    {
        EmailAddress = emailAddress;
        Subject = subject;
        CC = cc;
        AttachmentsCount = attachmentsCount;
    }
    //Перекрытие
    public override void SendNotification()
    {
        Console.WriteLine($"Email отправлен на {EmailAddress}: {MessageText} (Тема: {Subject}, CC: {CC}, Вложений: {AttachmentsCount}, HTML: {IsHtml})");
        IsRead = true;
        Status = "Email отправлен";
    }

    // Перегрузка
    public void SendNotification(string customSubject)
    {
        var old = Subject;
        Subject = customSubject;
        SendNotification();
        Subject = old;
    }

    public override string GetNotificationDetails()
    {
        return base.GetNotificationDetails() + $", Email: {EmailAddress}, Тема: {Subject}, CC: {CC}, Вложений: {AttachmentsCount}, HTML: {IsHtml}";
    }

    public void ForwardTo(EmailNotification other)
    {
        other.MessageText = this.MessageText;
        Console.WriteLine($"Уведомление переслано с {EmailAddress} на {other.EmailAddress}");
    }

    public void Log()
    {
        Console.WriteLine($"[LOG] Email: {EmailAddress}, Subject: {Subject}, CC: {CC}, Attachments: {AttachmentsCount}");
    }

    public void Archive()
    {
        Status = "Архивировано";
        Console.WriteLine($"Email {NotificationId} архивирован.");
    }

    // Новый метод
    public void AddTag(string tag)
    {
        if (!Tags.Contains(tag)) Tags.Add(tag);
    }
}

public class SMSNotification : Notification, ILoggable, IArchivable
{
    public string PhoneNumber { get; set; }
    public string Operator { get; set; }
    public int SmsLength { get; set; }
    public bool IsInternational { get; set; }

    // Новые методы
    public string CarrierCode { get; set; }
    public bool IsUnicode { get; set; }
    public string RegionCode { get; set; }

    public SMSNotification() { }

    public SMSNotification(int notificationId, string messageText, string phoneNumber, string operatorName, int smsLength, bool isInternational, string priority, string recipient, string sender, string status)
        : base(notificationId, messageText, "SMS", priority, recipient, sender, status)
    {
        PhoneNumber = phoneNumber;
        Operator = operatorName;
        SmsLength = smsLength;
        IsInternational = isInternational;
    }
    // Перекрытие
    public override void SendNotification()
    {
        Console.WriteLine($"SMS отправлено на {PhoneNumber}: {MessageText} (Оператор: {Operator}, Длина: {SmsLength}, Международный: {IsInternational}, Unicode: {IsUnicode})");
        IsRead = true;
        Status = "SMS отправлено";
    }

    // Перегрузка
    public void SendNotification(string countryCode)
    {
        PhoneNumber = countryCode + PhoneNumber;
        SendNotification();
    }
    //Перекрытие
    public override string GetNotificationDetails()
    {
        return base.GetNotificationDetails() + $", Телефон: {PhoneNumber}, Оператор: {Operator}, Длина: {SmsLength}, Международный: {IsInternational}, Unicode: {IsUnicode}";
    }

    public void CopyTo(SMSNotification other)
    {
        other.MessageText = this.MessageText;
        Console.WriteLine($"SMS уведомление скопировано с {PhoneNumber} на {other.PhoneNumber}");
    }

    public void Log()
    {
        Console.WriteLine($"[LOG] SMS: {PhoneNumber}, Operator: {Operator}, Length: {SmsLength}, International: {IsInternational}");
    }

    public void Archive()
    {
        Status = "Архивировано";
        Console.WriteLine($"SMS {NotificationId} архивировано.");
    }
}

public class PushNotification : Notification, ILoggable, IArchivable
{
    public string Platform { get; set; }
    public string AppName { get; set; }
    public string DeviceId { get; set; }
    public bool IsSilent { get; set; }

    // Новые атрибуты
    public int BadgeCount { get; set; }
    public string Sound { get; set; }
    public string Category { get; set; }

    public PushNotification() { }

    public PushNotification(int notificationId, string messageText, string platform, string appName, string deviceId, bool isSilent, string priority, string recipient, string sender, string status)
        : base(notificationId, messageText, "Push", priority, recipient, sender, status)
    {
        Platform = platform;
        AppName = appName;
        DeviceId = deviceId;
        IsSilent = isSilent;
    }

    public override void DisplayNotification()
    {
        Console.WriteLine($"Push на {Platform} ({AppName}, Device: {DeviceId}, Silent: {IsSilent}, Badge: {BadgeCount}): {MessageText}");
    }

    // Перегрузка
    public void DisplayNotification(string userLocale, bool showBadge)
    {
        var prefix = userLocale == "ru-RU" ? "[RU]" : "[EN]";
        Console.WriteLine($"{prefix} Push на {Platform} ({AppName}): {MessageText}");
        if (showBadge) Console.WriteLine($"Badge: {BadgeCount}");
    }
    // Перекрытие
    public override string GetNotificationDetails()
    {
        return base.GetNotificationDetails() + $", Платформа: {Platform}, Приложение: {AppName}, DeviceId: {DeviceId}, Silent: {IsSilent}, Badge: {BadgeCount}";
    }

    public void SyncWith(PushNotification other)
    {
        other.MessageText = this.MessageText;
        Console.WriteLine($"Push уведомление синхронизировано между {Platform} и {other.Platform}");
    }

    public void Log()
    {
        Console.WriteLine($"[LOG] Push: {Platform}, App: {AppName}, DeviceId: {DeviceId}, Silent: {IsSilent}");
    }

    public void Archive()
    {
        Status = "Архивировано";
        Console.WriteLine($"Push {NotificationId} архивировано.");
    }
}

public class EmailMarketingNotification : EmailNotification
{
    public string CampaignName { get; set; }
    public int TargetAudienceSize { get; set; }
    public bool IsAutomated { get; set; }
    public string MarketingChannel { get; set; }

    // Новые атрибуты
    public DateTime CampaignStart { get; set; }
    public DateTime CampaignEnd { get; set; }
    public double ConversionRate { get; set; } = 0.0;

    public EmailMarketingNotification(
        int notificationId,
        string messageText,
        string emailAddress,
        string subject,
        string cc,
        int attachmentsCount,
        string priority,
        string recipient,
        string sender,
        string status,
        string campaignName,
        int targetAudienceSize,
        bool isAutomated,
        string marketingChannel)
        : base(notificationId, messageText, emailAddress, subject, cc, attachmentsCount, priority, recipient, sender, status)
    {
        CampaignName = campaignName;
        TargetAudienceSize = targetAudienceSize;
        IsAutomated = isAutomated;
        MarketingChannel = marketingChannel;
    }
    // Перекрытие
    public override void SendNotification()
    {
        base.SendNotification();
        Console.WriteLine($"Маркетинговая кампания: {CampaignName}, Канал: {MarketingChannel}, Автоматизация: {IsAutomated}, Аудитория: {TargetAudienceSize}");
    }

    public void ScheduleCampaign(DateTime date)
    {
        CampaignStart = date;
        Console.WriteLine($"Кампания '{CampaignName}' запланирована на {date}");
    }

    public void AnalyzeResults()
    {
        ConversionRate = new Random().NextDouble() * 0.2; // простая имитация
        Console.WriteLine($"Анализ результатов кампании '{CampaignName}' завершен. ConversionRate={ConversionRate:P2}");
    }
}

// Generic класс
public class NotificationManager<T> where T : Notification
{
    private readonly List<T> _items = new List<T>();

    public void Add(T item)
    {
        _items.Add(item);
        Console.WriteLine($"Добавлено уведомление {item.NotificationId} ({item.Type})");
    }

    public bool Remove(int id)
    {
        var it = _items.FirstOrDefault(x => x.NotificationId == id);
        if (it == null) return false;
        _items.Remove(it);
        Console.WriteLine($"Удалено уведомление {id}");
        return true;
    }

    public T GetById(int id) => _items.FirstOrDefault(x => x.NotificationId == id);

    public IEnumerable<T> FilterByTag(string tag) => _items.Where(n => n.Tags.Contains(tag));

    public IEnumerable<T> FilterByRecipient(string recipient) => _items.Where(n => n.Recipient == recipient);

    public void SendAll(Func<T, bool> predicate = null)
    {
        var list = predicate == null ? _items : _items.Where(predicate);
        foreach (var n in list)
        {
            n.SendNotification();
        }
    }

    public void ArchiveAll()
    {
        foreach (var n in _items.OfType<IArchivable>())
        {
            ((IArchivable)n).Archive();
        }
    }
}

// Пример 
var email1 = new EmailNotification(1, "Проверьте вашу почту!", "user@example.com", "Входящее", "admin@site.com", 2, "Высокий", "user@example.com", "admin@site.com", "Создано");
var email2 = new EmailNotification(2, "Вторая почта", "admin@example.com", "Входящее", "user@site.com", 0, "Низкий", "admin@example.com", "user@site.com", "Создано");
email1.Tags.Add("important");
email1.Metadata["Folder"] = "Inbox";
email1.IsHtml = true;
email1.SendNotification();
email1.SendNotification("СПЕЦИАЛЬНАЯ ТЕМА");
Console.WriteLine(email1.GetNotificationDetails(true));
email1.ForwardTo(email2);
email1.Log();
email1.Archive();
Console.WriteLine("------------------------------------------------------------------");

var sms1 = new SMSNotification(3, "Ваш код: 1234", "+79991234567", "МТС", 12, false, "Средний", "+79991234567", "system", "Создано");
sms1.Tags.Add("auth");
sms1.IsUnicode = false;
sms1.SendNotification();
sms1.SendNotification("+7"); // overloaded
Console.WriteLine(sms1.GetNotificationDetails(true));
sms1.Log();
sms1.Archive();
Console.WriteLine("------------------------------------------------------------------");

var push1 = new PushNotification(5, "Новое обновление доступно!", "Android", "MyApp", "dev123", false, "Высокий", "dev123", "system", "Создано");
push1.BadgeCount = 3;
push1.DisplayNotification();
push1.DisplayNotification("ru-RU", true);
push1.SendNotification();
Console.WriteLine(push1.GetNotificationDetails(true));
push1.Log();
push1.Archive();
Console.WriteLine("------------------------------------------------------------------");

var marketingEmail = new EmailMarketingNotification(
    10, "Скидки только сегодня!", "client@site.com", "Акция!", "marketing@site.com", 1, "Высокий", "client@site.com", "marketing@site.com", "Создано",
    "Весенний Sale", 5000, true, "Email");

marketingEmail.ScheduleCampaign(DateTime.Now.AddDays(1));
marketingEmail.SendNotification();
marketingEmail.AnalyzeResults();
marketingEmail.Log();
marketingEmail.Archive();
Console.WriteLine("------------------------------------------------------------------");

// Generic использование
var manager = new NotificationManager<Notification>();
manager.Add(email1);
manager.Add(sms1);
manager.Add(push1);
manager.SendAll(n => n.Priority == "Высокий" || n.Tags.Contains("auth"));
manager.ArchiveAll();

Email отправлен на user@example.com: Проверьте вашу почту! (Тема: Входящее, CC: admin@site.com, Вложений: 2, HTML: True)
Email отправлен на user@example.com: Проверьте вашу почту! (Тема: СПЕЦИАЛЬНАЯ ТЕМА, CC: admin@site.com, Вложений: 2, HTML: True)
ID: 1, Тип: Email, Текст: Проверьте вашу почту!, Время: 10/19/2025 6:57:07 PM, Прочитано: True, Приоритет: Высокий, Получатель: user@example.com, Отправитель: admin@site.com, Статус: Email отправлен, Email: user@example.com, Тема: Входящее, CC: admin@site.com, Вложений: 2, HTML: True, Метаданные: Folder=Inbox
Уведомление переслано с user@example.com на admin@example.com
[LOG] Email: user@example.com, Subject: Входящее, CC: admin@site.com, Attachments: 2
Email 1 архивирован.
------------------------------------------------------------------
SMS отправлено на +79991234567: Ваш код: 1234 (Оператор: МТС, Длина: 12, Международный: False, Unicode: False)
SMS отправлено на +7+79991234567: Ваш код: 1234 (Оператор: МТС, Длина: 12, Международный: Fal